In [1]:
import os
import pandas as pd
import json
import numpy as np

In [2]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [3]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = 4

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONTYPE']

LABEL_COLUMN = 'Class_MotionType'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\\TEST2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TEST_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONTYPE-STEPS-V0.npy'
ALL_Y_TEST_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONTYPE-STEPS-V0.npy'

KeyError: 'STEPS_PITCH_ROTATION_THRESHOLD'

In [ ]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'LAR': 0, 'SML': 1}


In [ ]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "SIDESTEPS" not in file and "STEPS" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TEST2-STEPS-LR-LAR-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-132BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-184BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-196BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-264BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-46BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-49BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-65BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-66BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-90BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-92BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-98BPM.xlsx', 'PROC-TEST2-STEPS-LR-SML-100BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-SML-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-25BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-SML-50BPM.xlsx', 'PROC-TEST2-STEPS-LR-SML-65BPM-A

In [ ]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360

In [ ]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list_readings = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values_readings = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input_data_list_readings.append(row_values_readings)

    # turnn list into array to do arthimetic
    raw_sensor_data = np.array(input_data_list_readings)

    max_sensor_reading = np.max(raw_sensor_data, axis=0)

    input_data_array = np.copy(max_sensor_reading)

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [ ]:
ALL_X = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y = np.empty((0))  # List to store all Y training data

In [ ]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTest, yTest = df2Xy(df, WINDOW_SIZE)

        yTest = yTest.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTest])
        ALL_Y = np.concatenate([ALL_Y, yTest])
        
        print('xTest.shape:', xTest.shape)
        print('ALL_X.shape:', ALL_X.shape, ' ALL_Y.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)
    

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TEST_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TEST_CLASSIFY_PATH, numerical_label)


In [ ]:
processData(ALL_X, ALL_Y)

ALL_X_TEST_CLASSIFY = np.load(ALL_X_TEST_CLASSIFY_PATH)
ALL_Y_TEST_CLASSIFY = np.load(ALL_Y_TEST_CLASSIFY_PATH, allow_pickle=True)

print(ALL_X_TEST_CLASSIFY.shape)
print(ALL_Y_TEST_CLASSIFY.shape)

xTest.shape: (3771, 4)
ALL_X.shape: (3771, 4)  ALL_Y.shape: (3771,)
-------------------------------
xTest.shape: (3618, 4)
ALL_X.shape: (7389, 4)  ALL_Y.shape: (7389,)
-------------------------------
xTest.shape: (3582, 4)
ALL_X.shape: (10971, 4)  ALL_Y.shape: (10971,)
-------------------------------
xTest.shape: (3488, 4)
ALL_X.shape: (14459, 4)  ALL_Y.shape: (14459,)
-------------------------------
xTest.shape: (3501, 4)
ALL_X.shape: (17960, 4)  ALL_Y.shape: (17960,)
-------------------------------
xTest.shape: (3771, 4)
ALL_X.shape: (21731, 4)  ALL_Y.shape: (21731,)
-------------------------------
xTest.shape: (3618, 4)
ALL_X.shape: (25349, 4)  ALL_Y.shape: (25349,)
-------------------------------
xTest.shape: (7183, 4)
ALL_X.shape: (32532, 4)  ALL_Y.shape: (32532,)
-------------------------------
xTest.shape: (6995, 4)
ALL_X.shape: (39527, 4)  ALL_Y.shape: (39527,)
-------------------------------
xTest.shape: (7021, 4)
ALL_X.shape: (46548, 4)  ALL_Y.shape: (46548,)
----------------

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [ ]:
print(ALL_X_TEST_CLASSIFY.shape)
print(ALL_Y_TEST_CLASSIFY.shape)


(125012, 4)
(125012,)
